<div class="alert alert-info">
<h2> Комментарий от ревьюера</h2>
Привет. Меня зовут Алина Гусева, я буду делать ревью к твоей работе по статистическому анализу данных. В дальнейшем предлагаю общаться на "ты".
    
Видно, что ты старался, делая проект. Все ключевые этапы проекта выполнены, в целом тебе удалось со всем справиться. Особенно хочется отметить, что соблюдена структура проекта, а код написан аккуратно. Есть, правда, несколько замечаний, которые нужно исправить.
    
Для того, чтобы и тебе и мне удобнее бы
    ло ориентироваться в комментариях, я буду помечать их разными цветами:
- красный цвет означает комментарии, которые требуют доработки. Как только ты их исправишь - проект будет принят;
- желтый цвет - то, что в следующий раз можно будет сделать по-другому. На твое усмотрение можешь доделать маленькие правки сейчас или учесть замечание для выполнения следующих проектов;
- зеленый цвет - очень элегантные и удачные решение, которые желательно использовать в дальнейшей работе.
- синий цвет - информация
    
Предлагаю работать над проектом в диалоге: если ты что-то изменяешь или вносишь какие-то коррективы - выделяй это каким-нибудь цветом и пиши об этом мне. Так мне гораздо легче будет отследить сделанные изменения.
    
Также попрошу тебя не изменять и не удалять мои комментарии, чтобы проверка твоего самостоятельного проекта прошла оперативнее.
    
 
</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [5]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')
print(data.info())
print(data.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1  

### Вывод

Таблица состоит из 12 столбцов и 21525 строк.
Заметили пропуски значений в days_employed и total_income.
Проанализировали типы данных во всех столбцах и их  количество: float64(2), int64(5), object(5)

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Молодец, хороший тон использовать метод info() для получения общей информации из таблицы.
 Также верно сделаны выводы в контексте данного задания.

</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
print(len(data['days_employed'].unique()))


19352


В столбце стажа было найдено 19352 уникальных значения.
Столбец имеет числовой тип данных, поэтому мы может его отсортировать и посмотреть на значения в концовке.

In [7]:
print(data.sort_values(by='days_employed').tail(5))

       children  days_employed  dob_years education  education_id  \
21489         2            NaN         47   Среднее             1   
21495         1            NaN         50   среднее             1   
21497         0            NaN         48    ВЫСШЕЕ             0   
21502         1            NaN         42   среднее             1   
21510         2            NaN         28   среднее             1   

          family_status  family_status_id gender income_type  debt  \
21489   женат / замужем                 0      M   компаньон     0   
21495  гражданский брак                 1      F   сотрудник     0   
21497   женат / замужем                 0      F   компаньон     0   
21502   женат / замужем                 0      F   сотрудник     0   
21510   женат / замужем                 0      F   сотрудник     0   

       total_income                           purpose  
21489           NaN              сделка с автомобилем  
21495           NaN                           свадьб

Видим, что в столбце стажа есть значения NaN. Это количественная переменная, поэтому можем оценить несколько вариантов:
Сколько пропусков относительно всей таблицы? Если меньше 1 процента, то можно удалить эти строки.
Если нет, то подставить либо среднее, либо медиану. Оценить что более логично поможет для отчета.

In [8]:
print('В столбце со стажем пропущено {:.1%} значений'.format(data['days_employed'].isna().sum()/data['days_employed'].count()))

В столбце со стажем пропущено 11.2% значений


Удалять нельзя, нужно заполнять.
Пропуски в этом столбце могли появиться из-за того, что клиенты могли не указывать свой стаж работы.

In [9]:
print('Среднее по стажу: {:.2f}'.format(data['days_employed'].mean()))
print('Медиана по стажу: {:.2f}'.format(data['days_employed'].median()))

Среднее по стажу: 63046.50
Медиана по стажу: -1203.37


Среднее значение в данном случае не отображает верную информацию. У кого-то может быть огромный стаж и это может портить выборку. Среднее сильно завышено от реально возможных значений. Это тревожит, нужно проверить.

Также замечаю, что медиана по стажу у меня отрицательная. То есть в большинстве случаев у меня указан отрицательный стаж.
Нужно оценить сколько таких значений, возможно ошибка при выгрузке.


In [10]:
print('Процент отрицательного стажа: {:.1%}'.format(data[data['days_employed']<0]['days_employed'].count()/data['days_employed'].count()))

Процент отрицательного стажа: 82.2%


Меняю знак для отрицательного стажа. Ошибка могла возникнуть из-за разной логики слияния данных или при выгрузке.

In [11]:
data['days_employed']=abs(data['days_employed'])
print('Процент отрицательного стажа: {:.1%}'.format(data[data['days_employed']<0]['days_employed'].count()/data['days_employed'].count()))

Процент отрицательного стажа: 0.0%


In [12]:
print('Средний стаж одного человека в годах: {:.0f}'.format(data['days_employed'].mean()/365))

Средний стаж одного человека в годах: 183


Такого быть не может, оценим сколько таких значений в нашей таблице. Возьмем возраст каждого заёмщика и посчитаем какой максимальный стаж в днях у него может быть на данный момент начиная с 18 лет.

In [13]:
print('Процент завышенного стажа: {:.1%}'.format(data[(data['days_employed']/365)>(data['dob_years']-18)]['days_employed'].count()/data['days_employed'].count()))

Процент завышенного стажа: 20.3%


На этом моменте было множество рассуждений и задан вопрос в канале проектов. Оказалось, что далее нам не требуется использовать этот столбец, поэтому осталяю так.
Избавились от артефактов и возвращаемся к подсчёту нового среднего и медианы.

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Да, это действительно так. По какой-то причине ланные грузятся некорректно и в  таблице возникает очень много отрицательных значеий. Естесственно, такого быть не должно. Прежде всего нужно обработать их, а потом исктаь пути решения. Тебе верно подсказли, что для дальнейшего исследвания столбец не пригодится. В таком случае, можно сделать минимальную предобработку и сообщить о некорректности данных разработчику баз данных.

</div>

In [14]:
print('Правильное среднее по стажу: {:.2f}'.format(data['days_employed'].mean()))
print('Правильная медиана по стажу: {:.2f}'.format(data['days_employed'].median()))

Правильное среднее по стажу: 66914.73
Правильная медиана по стажу: 2194.22


При большом разбросе медиана более корректно отображает значение стажа

In [15]:
data['days_employed']=data['days_employed'].fillna(data['days_employed'].median())
data['days_employed'].count()

21525

Замена прошла успешно, снова проверим среднее и медиану после замены.

In [16]:
print('После замены седнее по просрочкам: {:.2f}'.format(data['days_employed'].mean()))
print('После замены медиана по просрочкам: {:.2f}'.format(data['days_employed'].median()))

После замены седнее по просрочкам: 60378.03
После замены медиана по просрочкам: 2194.22


Среднее логичным образом снизилось. Медиана не изменилась.
Переходим с столбцу с доходами.

In [17]:
print(len(data['total_income'].unique()))

19352


В столбце доходов было найдено 19352 уникальных значения.
Столбец имеет числовой тип данных, поэтому мы может его отсортировать и посмотреть на значения в концовке.

In [18]:
print(data.sort_values(by='total_income').tail(5))

       children  days_employed  dob_years education  education_id  \
21489         2    2194.220567         47   Среднее             1   
21495         1    2194.220567         50   среднее             1   
21497         0    2194.220567         48    ВЫСШЕЕ             0   
21502         1    2194.220567         42   среднее             1   
21510         2    2194.220567         28   среднее             1   

          family_status  family_status_id gender income_type  debt  \
21489   женат / замужем                 0      M   компаньон     0   
21495  гражданский брак                 1      F   сотрудник     0   
21497   женат / замужем                 0      F   компаньон     0   
21502   женат / замужем                 0      F   сотрудник     0   
21510   женат / замужем                 0      F   сотрудник     0   

       total_income                           purpose  
21489           NaN              сделка с автомобилем  
21495           NaN                           свадьб

Видим, что в столбце дохода есть значения NaN. Это количественная переменная, поэтому можем оценить несколько вариантов:
Сколько пропусков относительно всей таблицы? Если меньше 1 процента, то можно удалить эти строки.
Если нет, то подставить либо среднее, либо медиану. Оценить что более логично поможет для отчета.

In [19]:
print('В столбце с доходами пропущено {:.1%} значений'.format(data['total_income'].isna().sum()/data['total_income'].count()))

В столбце с доходами пропущено 11.2% значений


Удалять нельзя, нужно заполнять. 
Пропуски в этом столбце могли появиться из-за того, что данные по доходам могли быть просто не предоставлены.

In [20]:
print('Среднее по доходам: {:.2f}'.format(data['total_income'].mean()))
print('Медиана по доходам: {:.2f}'.format(data['total_income'].median()))

Среднее по доходам: 167422.30
Медиана по доходам: 145017.94


Среднее значение в данном случае не отображает верную информацию. У кого-то может быть огромный доход и это может портить выборку.
Делаю выбор в пользу медианы.

In [21]:
data['total_income']=data['total_income'].fillna(data['total_income'].median())
data['total_income'].count()

21525

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
В принципе, у тебя верный подход. Но для более точной замены можно было предварительно сгруппировать данные(например, по типу занятости) и уже относительно этих групп заменять пропуски медианным или средним значением. Ведь зарплата у студента и госсслужащего отличается значительно, а сейчас мы все причешем под одну гребенку.
</div> 

In [22]:
print('После замены седнее по доходам: {:.2f}'.format(data['total_income'].mean()))
print('После замены медиана по доходам: {:.2f}'.format(data['total_income'].median()))

После замены седнее по доходам: 165159.49
После замены медиана по доходам: 145017.94


Среднее логичным образом снизилось. Медиана не изменилась.
Проверям информацию о всей таблице.

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Земена пропусков прошла успешно, данные не пострадали. Все столбцы заполнены.

Возможно было бы логичнее разделить медианы по типам в соответствии с возрастной группой, образованием и типом занятости. Такие выборки дали бы более точные значения медиан, но таких вариантов (если разделить хотя бы на два значения в каждом виде) получилось бы минимум 8.

Если задача потсавлена в короткие сроки, то это можно пропустить.

### Замена типа данных

Нас попросили заменить вещественный тип на целый. Таких столбцов у нас всего два и для замены идеально подойдёт метод astype()
Другие методы нет смысла использовать, потому что нам нужно отбросить только дробную часть числа. Это не строковый тип, и не временной. 

In [24]:
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

После замены типа с вещественного на целочисленный у нас в таблице осталось только два типа данных. 7 столбцов с целыми числами, 5 со строками 

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Молодец, все столбцы имееют нужный тип данных, можно приступать к следующему шагу)
</div>

### Обработка дубликатов

У нас есть 5 столбцов со строковым типом данных. Нужно проверить их на потенциальные дубликаты. 

In [25]:
from collections import Counter
Counter(data['education'])

Counter({'высшее': 4718,
         'среднее': 13750,
         'Среднее': 711,
         'СРЕДНЕЕ': 772,
         'ВЫСШЕЕ': 274,
         'неоконченное высшее': 668,
         'начальное': 250,
         'Высшее': 268,
         'НЕОКОНЧЕННОЕ ВЫСШЕЕ': 29,
         'Неоконченное высшее': 47,
         'НАЧАЛЬНОЕ': 17,
         'Начальное': 15,
         'Ученая степень': 1,
         'УЧЕНАЯ СТЕПЕНЬ': 1,
         'ученая степень': 4})

Очевидные дубли, которые отличаются только регистром. Возможно эта графа заполняется вручную пользователем, что может привести к ошибкам такого вида. Приведём все столбцы к нижнему регистру и проверим на уникальность.

In [26]:
data['education']=data['education'].str.lower()
Counter(data['education'])

Counter({'высшее': 5260,
         'среднее': 15233,
         'неоконченное высшее': 744,
         'начальное': 282,
         'ученая степень': 6})

Теперь всё хорошо

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Вот это замечание очень важно. В дальнейшем нужно не забывать приводить текстовый формат данных к единому виду

</div>

In [27]:
Counter(data['family_status'])

Counter({'женат / замужем': 12380,
         'гражданский брак': 4177,
         'вдовец / вдова': 960,
         'в разводе': 1195,
         'Не женат / не замужем': 2813})

In [28]:
Counter(data['gender'])

Counter({'F': 14236, 'M': 7288, 'XNA': 1})

Был найден случайный пропуск, который можно легко восстановить зная Фамилию или Имя. У нас таких данных нет. Найдём и посмотрим на эту строку.

In [29]:
print(data[data['gender']=='XNA'])

       children  days_employed  dob_years            education  education_id  \
10701         0           2358         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

       total_income               purpose  
10701        203905  покупка недвижимости  


Была единственная надежда на декрет. Но его здесь нет. Придётся удалить эту строку.

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Ошибка в данных, такое бывает. Конечно, такую строчку нужно удалять
</div>

In [30]:
data.drop(data[data['gender']=='XNA'].index, inplace = True) 
data.reset_index(drop=True)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None


In [31]:
Counter(data['gender'])

Counter({'F': 14236, 'M': 7288})

Удаление произошло корректно

In [32]:
Counter(data['income_type'])

Counter({'сотрудник': 11119,
         'пенсионер': 3856,
         'компаньон': 5084,
         'госслужащий': 1459,
         'безработный': 2,
         'предприниматель': 2,
         'студент': 1,
         'в декрете': 1})

In [33]:
purpose_count=Counter(data['purpose'])
purpose_count

Counter({'покупка жилья': 647,
         'приобретение автомобиля': 462,
         'дополнительное образование': 462,
         'сыграть свадьбу': 774,
         'операции с жильем': 653,
         'образование': 447,
         'на проведение свадьбы': 777,
         'покупка жилья для семьи': 641,
         'покупка недвижимости': 623,
         'покупка коммерческой недвижимости': 664,
         'покупка жилой недвижимости': 607,
         'строительство собственной недвижимости': 635,
         'недвижимость': 634,
         'строительство недвижимости': 620,
         'на покупку подержанного автомобиля': 479,
         'на покупку своего автомобиля': 505,
         'операции с коммерческой недвижимостью': 651,
         'строительство жилой недвижимости': 626,
         'жилье': 647,
         'операции со своей недвижимостью': 630,
         'автомобили': 478,
         'заняться образованием': 412,
         'сделка с подержанным автомобилем': 489,
         'получение образования': 443,
         'авт

In [34]:
print(Counter(data['children']))

Counter({0: 14148, 1: 4818, 2: 2055, 3: 330, 20: 76, -1: 47, 4: 41, 5: 9})


Были найдены артефакты с отрицательным количетсвом детей, их меньше 1 процента - можно удалить.

<div class="alert alert-warning">
<h2> Комментарий от ревьюера</h2>
Также подозрительно много пар с количеством детей 20. При этом нет людей с 19,18,17... Скорее всего здесь тоже ошибка и значение необходимо поделить на 10.

</div>

In [35]:
data.drop(data[data['children']<0].index, inplace = True) 
data.reset_index(drop=True)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21477 entries, 0 to 21524
Data columns (total 12 columns):
children            21477 non-null int64
days_employed       21477 non-null int64
dob_years           21477 non-null int64
education           21477 non-null object
education_id        21477 non-null int64
family_status       21477 non-null object
family_status_id    21477 non-null int64
gender              21477 non-null object
income_type         21477 non-null object
debt                21477 non-null int64
total_income        21477 non-null int64
purpose             21477 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None


### Вывод

В нашей таблице был только один строковый столбец заполнен ошибочно. Привели всё к одному регистру и проблема решилась. Также была найдена строка со случайной ошибкой в столбце пол, восстановить данные не удалось, удалили её.

### Лемматизация

In [36]:
from pymystem3 import Mystem
m = Mystem()
purpose_lemma=[] #создал пустой список для накопления всех слов
for i in data['purpose']:
    for word in i.split():
        if word!=' ':
            purpose_lemma+=(m.lemmatize(word))
print(purpose_lemma[:10])

['покупка', '\n', 'жилье', '\n', 'приобретение', '\n', 'автомобиль', '\n', 'покупка', '\n']


После лемматизации получил список, посчитаем количество разных слов в приведенном списке

In [37]:
purpose_lemma.sort()
Counter(purpose_lemma)

Counter({'\n': 55085,
         'автомобиль': 4304,
         'высокий': 1374,
         'для': 1293,
         'дополнительный': 906,
         'жилой': 1228,
         'жилье': 4465,
         'заниматься': 908,
         'коммерческий': 1314,
         'на': 2229,
         'недвижимость': 6350,
         'образование': 4012,
         'операция': 2603,
         'подержать': 966,
         'покупка': 5898,
         'получение': 1314,
         'приобретение': 460,
         'проведение': 776,
         'профильный': 434,
         'ремонт': 611,
         'с': 2919,
         'свадьба': 2346,
         'свой': 2228,
         'сдача': 652,
         'сделка': 943,
         'семья': 641,
         'со': 627,
         'собственный': 634,
         'строительство': 1877,
         'сыграть': 773})

### Вывод

Среди целей кредита чаще всего встречаются слова 'покупка','жилье','автомобиль','образование','недвижимость' и 'свадьба'.
Это исследование поможет нам разделить наши цели всего на несколько типов.

### Категоризация данных

In [38]:
from pymystem3 import Mystem
m = Mystem()
def lem(row):
    l=m.lemmatize(row)
    if 'автомобиль' in l:
        return 'автомобиль'
    elif 'свадьба' in l:
        return 'свадьба'
    elif 'образование' in l:
        return 'образование'
    elif 'жилье' or 'недвижимость' in l:
        return 'недвижимость'
data['purpose_type']=data['purpose'].apply(lem)
data['purpose_type'].value_counts()

недвижимость    10815
автомобиль       4304
образование      4012
свадьба          2346
Name: purpose_type, dtype: int64

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Ура, лемматизация прошла успешно, можно двигаться к следующему шагу)

</div>

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21477 entries, 0 to 21524
Data columns (total 13 columns):
children            21477 non-null int64
days_employed       21477 non-null int64
dob_years           21477 non-null int64
education           21477 non-null object
education_id        21477 non-null int64
family_status       21477 non-null object
family_status_id    21477 non-null int64
gender              21477 non-null object
income_type         21477 non-null object
debt                21477 non-null int64
total_income        21477 non-null int64
purpose             21477 non-null object
purpose_type        21477 non-null object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


Добавили новый столбец, после применения функции он оказался полностью заполнен

### Вывод

Лемматизация позволила нам разделить цели получения кредита всего на 4 категории. Что позволит в дальнейшем оценить как цел влияет на задолженность.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [40]:
print('Процент должников по всей таблице: {:.0%}'.format(data['debt'].sum()/data['debt'].count()))

Процент должников по всей таблице: 8%


In [41]:
print('Должники сгруппированные по детям: \n',data.groupby('children')['debt'].count())

Должники сгруппированные по детям: 
 children
0     14148
1      4818
2      2055
3       330
4        41
5         9
20       76
Name: debt, dtype: int64


В нашей таблице количество детей между клиентами распределенно неравномерно. ПОэтому будет логично проверять каждого внутри своей категории.

In [42]:
children_unique=data['children'].unique()
children_unique.sort()
for i in children_unique:
    print('Процент просрочек с',i,'детьми: {:.2%}'.format(data[data['children']==i]['debt'].sum()/data[data['children']==i]['debt'].count()))

Процент просрочек с 0 детьми: 7.51%
Процент просрочек с 1 детьми: 9.22%
Процент просрочек с 2 детьми: 9.44%
Процент просрочек с 3 детьми: 8.18%
Процент просрочек с 4 детьми: 9.76%
Процент просрочек с 5 детьми: 0.00%
Процент просрочек с 20 детьми: 10.53%


### Вывод

Исследование внутри каждой группы показало, что процент просрочки прмерно совпадает с общим процентом просрочек. Наблюдается небольшой рост при увеличении количества детей.

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Да, заемщики без детей самые отвтственные.
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
print('Должники сгруппированные по семейному положению: \n',data.groupby('family_status')['debt'].count())

Должники сгруппированные по семейному положению: 
 family_status
Не женат / не замужем     2808
в разводе                 1191
вдовец / вдова             956
гражданский брак          4171
женат / замужем          12351
Name: debt, dtype: int64


В нашей таблице семейные статус между клиентами распределен неравномерно. ПОэтому будет логично проверять каждого внутри своей категории.

In [44]:
family_status_unique=data['family_status'].unique()
family_status_unique.sort()
for i in family_status_unique:
    print('Процент просрочек с семейным статусом',i,': {:.2%}'.format(data[data['family_status']==i]['debt'].sum()/data[data['family_status']==i]['debt'].count()))

Процент просрочек с семейным статусом Не женат / не замужем : 9.76%
Процент просрочек с семейным статусом в разводе : 7.14%
Процент просрочек с семейным статусом вдовец / вдова : 6.59%
Процент просрочек с семейным статусом гражданский брак : 9.30%
Процент просрочек с семейным статусом женат / замужем : 7.53%


### Вывод

Исследование внутри каждой группы показало, что процент просрочки прмерно совпадает с общим процентом просрочек. С положительной динамикой выделяются вдовцы, затем разведенные и женатые. Больше всего просрочек у не женатых или живущих в гражданском браке.

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Молодец, вывод сделан правильный!

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
print('Максимальная зарплата: {:.0f}'.format(data['total_income'].max()))
print('Средняя зарплата: {:.0f}'.format(data['total_income'].mean()))
print('Медиана по зарплате: {:.0f}'.format(data['total_income'].median()))
print('Минимальная зарплата: {:.0f}'.format(data['total_income'].min()))

Максимальная зарплата: 2265604
Средняя зарплата: 165183
Медиана по зарплате: 145017
Минимальная зарплата: 20667


Оценив разброс зарплат можно разделить всех клиентов на несколько категорий.

In [46]:
def income(x):
    if x<50000 :
        return 'низкий'
    elif x<100000:
        return 'средний'
    elif x<150000:
        return 'выше среднего'
    elif x<200000:
        return 'выcокий'
    else:
        return 'очень выcокий'
data['income_level']=data['total_income'].apply(income)
data['income_level'].value_counts()

выше среднего    7858
очень выcокий    5057
выcокий          4107
средний          4084
низкий            371
Name: income_level, dtype: int64

В нашей таблице уровень дохода между клиентами распределен неравномерно. ПОэтому будет логично проверять каждого внутри своей категории.

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>
Здесь все зависит от того, как ты разделишь на группы. Лучше всего использовать квантиль, чтобы группы получились равные по количеству людей.
</div>

In [47]:
income_level_unique=['низкий','средний','выше среднего','выcокий','очень выcокий']
for i in income_level_unique:
    print('Процент просрочек клиентов, у которых уровень дохода',i,': {:.2%}'.format(data[data['income_level']==i]['debt'].sum()/data[data['income_level']==i]['debt'].count()))

Процент просрочек клиентов, у которых уровень дохода низкий : 6.20%
Процент просрочек клиентов, у которых уровень дохода средний : 8.10%
Процент просрочек клиентов, у которых уровень дохода выше среднего : 8.40%
Процент просрочек клиентов, у которых уровень дохода выcокий : 8.96%
Процент просрочек клиентов, у которых уровень дохода очень выcокий : 7.08%


### Вывод

На удивление, но больше всего просрочек у людей с высоким уровнем дохода. У нас наблюдается обратная зависимость. Исключение - клиенты с очень высоким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [48]:
purpose_type_unique=data['purpose_type'].unique()
purpose_type_unique.sort()
for i in purpose_type_unique:
    print('Процент клиентов, у которых целью кредита является',i,': {:.2%}'.format(data[data['purpose_type']==i]['debt'].sum()/data[data['purpose_type']==i]['debt'].count()))

Процент клиентов, у которых целью кредита является автомобиль : 9.34%
Процент клиентов, у которых целью кредита является недвижимость : 7.23%
Процент клиентов, у которых целью кредита является образование : 9.22%
Процент клиентов, у которых целью кредита является свадьба : 7.93%


### Вывод

Меньше просрочек бывает по направлению недвижимости и свадьбы, больше с образованием и автомобилями.

### Шаг 4. Общий вывод

На входе мы получили большую таблицу, в которой были пропуски и дубли. В таблице оказалось много артефактов, такие как отрицательный стаж, отрицательное количество детей, огромный стаж в несколько сотен лет. После небольшой обработки данных, удаления дублей и лемматизации целей мы смогли категоризировать данные и ответить на поставленные вопросы максимально точно. В ходе выполнения мною было добавлено несколько столбцов, в которые я сохранял категории. 

Конструкцию try-expect логичнее было бы использовать для перевода типа из 'объект' в 'число'

В моей таблице ещё остались артефакты, такие как огромный стаж.

In [49]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,очень выcокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,очень выcокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выcокий


<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>

Поздравляю с первым успешным проектом!

- Ты хорошо делаешь выводы и анализируешь результаты.
    
- Код написан аккуратно и качественно, видно, что есть какой-то определенный бэкграунд.

- Соблюдена структура проекта, было приятно проверять:)
    
- Все написано четко и по делу

    
 Желаю дальнейших успехов!


</div>


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.